Librerías y Funciones propias:

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..')) 

import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
import pandas as pd
from algoritmos.CC import timeAveragedcrossCorrelation
from algoritmos.GCC import generalizedCrossCorrelation
from algoritmos.DOA import doa
from helpers.obtainRoomDimensionsFromVolume import obtainRoomDimensionsFromVolume
from helpers.addNoise import addNoise

Señales de entrada:

In [2]:

fs, sourceSignal = wavfile.read("../audios/p262_001.wav")
sourceSignal = addNoise(sourceSignal, 30)

if sourceSignal.dtype != np.float32 and sourceSignal.dtype != np.float64:
    sourceSignal = sourceSignal.astype(np.float32)

if np.max(np.abs(sourceSignal)) > 1:
    sourceSignal = sourceSignal / np.max(np.abs(sourceSignal))

Creamos los rooms:

In [3]:
rt60TargetsDf = pd.read_excel("../datos-resultados/Datos.xlsx", engine='openpyxl', header=None)
rt60Targets = rt60TargetsDf.iloc[11, 4:105].to_numpy()

volumesDf = pd.read_excel("../datos-resultados/Datos.xlsx", engine='openpyxl', header=None)
volumes = volumesDf.iloc[10, 4:105].to_numpy()

roomsRt60 = []

for rt60Target in rt60Targets:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialMicPosition = [3.775, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics, 
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsRt60.append(room)


roomsVolume = []

for volume in volumes:

    # Room dimensions (width * length)
    roomDimensions = obtainRoomDimensionsFromVolume(volume)

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=48000, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, 1.5]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialMicPosition = [3.775, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics, 
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsVolume.append(room)

Simulamos los rooms

In [4]:

micSignalsArraysRt60 = []

for room in roomsRt60:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysRt60.append(micSignals)

micSignalsArraysVolume = []

for room in roomsVolume:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysVolume.append(micSignals)

# Revisar un room simulado
room = roomsRt60[0]

print("Sample rate:", room.fs)
print("Duración de la señal (s):", len(room.sources[0].signal) / room.fs)
print("Forma de micSignals:", micSignalsArraysRt60[0].shape)

for i in range(4):
    mic = micSignalsArraysRt60[0][i]
    print(f"Mic {i} - max: {np.max(np.abs(mic))}, min: {np.min(mic)}")

Sample rate: 48000
Duración de la señal (s): 2.348520833333333
Forma de micSignals: (4, 153544)
Mic 0 - max: 0.42539233804869936, min: -0.34078454546746956
Mic 1 - max: 0.37849518255706016, min: -0.3305972480905794
Mic 2 - max: 0.35458203669416977, min: -0.35458203669416977
Mic 3 - max: 0.36270689735783246, min: -0.36270689735783246


Algoritmos

Time Averaged Cross Correlation

In [ ]:
distanceBetweenMics = [0.15, 0.30, 0.45, 0.15, 0.30, 0.15]

doaCcRt60 = []

for micSignals in micSignalsArraysRt60:
    delays = []

    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[1], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[2], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[1], micSignals[2], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[1], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[2], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

    doaCcRt60.append(doa(delays, distanceBetweenMics))

doaCcVolume = []

for micSignals in micSignalsArraysVolume:
    delays = []
    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[1], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[2], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[0], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[1], micSignals[2], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[1], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)
    delays.append((timeAveragedcrossCorrelation(micSignals[2], micSignals[3], returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

    doaCcVolume.append(doa(delays, distanceBetweenMics))


dfRT60 = pd.DataFrame(doaCcRt60, columns=['DOA_CC_RT60'])
dfVolume = pd.DataFrame(doaCcVolume, columns=['DOA_CC_Volume'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfRT60.to_excel(writer, sheet_name='DOA_CC_RT60', index=False)
    dfVolume.to_excel(writer, sheet_name='DOA_CC_Volume', index=False)

KeyboardInterrupt: 

GCC: Classic

In [ ]:
doaClassicRt60 = []

for micSignals in micSignalsArraysRt60:
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicRt60.append(doa(delays, distanceBetweenMics))

doaClassicVolume = []

for micSignals in micSignalsArraysVolume:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicVolume.append(doa(delays, distanceBetweenMics))


dfRT60 = pd.DataFrame(doaClassicRt60, columns=['DOA_Classic_RT60'])
dfVolume = pd.DataFrame(doaClassicVolume, columns=['DOA_Classic_Volume'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfRT60.to_excel(writer, sheet_name='DOA_Classic_RT60', index=False)
    dfVolume.to_excel(writer, sheet_name='DOA_Classic_Volume', index=False)

Delays between microphones (in seconds): [0.0002916666666666667, 0.0005, 0.0007916666666666666, 0.0002916666666666667, 0.00047916666666666664, 0.00025]
DOA classic cross correlation:  52.71141540833792
Delays between microphones (in seconds): [0.0002916666666666667, 0.0005, 0.0007916666666666666, 0.0002916666666666667, 0.00047916666666666664, 0.00025]
DOA classic cross correlation:  52.71141540833792
Delays between microphones (in seconds): [0.0002916666666666667, 0.0005, 0.0007916666666666666, 0.0002916666666666667, 0.00047916666666666664, 0.00025]
DOA classic cross correlation:  52.71141540833792
Delays between microphones (in seconds): [0.0002916666666666667, 0.0005, 0.0007916666666666666, 0.0002916666666666667, 0.00047916666666666664, 0.00025]
DOA classic cross correlation:  52.71141540833792
Delays between microphones (in seconds): [0.0002916666666666667, 0.0005, 0.0007916666666666666, 0.00025, 0.00047916666666666664, 0.00025]
DOA classic cross correlation:  53.87227238870285
Dela

GCC: Roth

In [ ]:
doaRothRt60 = []

for micSignals in micSignalsArraysRt60:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothRt60.append(doa(delays, distanceBetweenMics))

doaRothVolume = []

for micSignals in micSignalsArraysVolume:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothVolume.append(doa(delays, distanceBetweenMics))


dfRT60 = pd.DataFrame(doaRothRt60, columns=['DOA_Roth_RT60'])
dfVolume = pd.DataFrame(doaRothVolume, columns=['DOA_Roth_Volume'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfRT60.to_excel(writer, sheet_name='DOA_Roth_RT60', index=False)
    dfVolume.to_excel(writer, sheet_name='DOA_Roth_Volume', index=False)

Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Roth:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Roth:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Roth:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Roth:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Roth:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.000291666666666666

GCC: SCOT

In [ ]:
doaScotRt60 = []

for micSignals in micSignalsArraysRt60:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotRt60.append(doa(delays, distanceBetweenMics))

doaScotVolume = []

for micSignals in micSignalsArraysVolume:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotVolume.append(doa(delays, distanceBetweenMics))


dfRT60 = pd.DataFrame(doaScotRt60, columns=['DOA_SCOT_RT60'])
dfVolume = pd.DataFrame(doaScotVolume, columns=['DOA_SCOT_Volume'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfRT60.to_excel(writer, sheet_name='DOA_SCOT_RT60', index=False)
    dfVolume.to_excel(writer, sheet_name='DOA_SCOT_Volume', index=False)

Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA SCOT:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA SCOT:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA SCOT:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA SCOT:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA SCOT:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.000291666666666666

GCC: Phase

In [ ]:
doaPhaseRt60 = []

for micSignals in micSignalsArraysRt60:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseRt60.append(doa(delays, distanceBetweenMics))

doaPhaseVolume = []

for micSignals in micSignalsArraysVolume:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseVolume.append(doa(delays, distanceBetweenMics))


dfRT60 = pd.DataFrame(doaPhaseRt60, columns=['DOA_Phase_RT60'])
dfVolume = pd.DataFrame(doaPhaseVolume, columns=['DOA_Phase_Volume'])

with pd.ExcelWriter('../datos-resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfRT60.to_excel(writer, sheet_name='DOA_Phase_RT60', index=False)
    dfVolume.to_excel(writer, sheet_name='DOA_Phase_Volume', index=False)

Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Phase:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Phase:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Phase:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Phase:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666666667]
DOA Phase:  45.55457333232342
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.0006041666666666667, 0.0002916666666